# Week 9 -- Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Functions

In [3]:
def check_names(df1, df2):
    """
    This funciton compares the list of names from one dataframe to another and updates where the names do not match 
    so they will merge correctly.
    
    Parameters:
        df1: first dataframe to compare to; this should be the dataframe on the left that you will join with when 
        they merge.
        df2: second dataframe, should be the dataframe on the right for the later merge.
        
    Returns:
        Updated name list for df2 that should match df1.
    """
    
    df_players = list(df1['Player'])    
    for i in range(len(df2['Player'])):
        name = df2['Player'][i]
        split_name = name.split(' ')
        if (split_name[0] + ' ' + split_name[1]) in df_players:
            df2['Player'][i] = split_name[0] + ' ' + split_name[1]
        else:
            df2['Player'][i] = name

In [4]:
def name_update(dataframe, column):
    """
    This function updates a column in a dataframe by stripping any excess spaces surrounding the observation.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column: the column you want to update.
    
    Returns:
        Updated dataframe column.    
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip())

In [5]:
def update_int_dtype(dataframe, column_list):
    """
    This function updates the data type for a list of columns to integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column data types to integer.   
    """
    for column in column_list:
        dataframe[column] = dataframe[column].map(lambda x: int(x))

In [6]:
def update_float_dtype(dataframe, column_list):
    """
    This function takes a dataframe object and converts it to a float.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
    
    Returns:
        Updated dataframe column data types to a float.  
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = '0.0'
            else:
                dataframe[column][i] = item 
            updated_item = str(dataframe[column][i])
            ones, tenths = updated_item.split('.')
            ones = int(ones)
            tenths = int(tenths) * .1
            dataframe[column][i] = ones + tenths
        dataframe[column] = dataframe[column].map(lambda x: float(x))

In [7]:
def fill_blanks(dataframe, column_list):
    """
    This function fills blank values with 0. This should be used for a column that will be an integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.    
    
    Returns:
        Updated dataframe columns with filled values.      
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = 0
            else:
                dataframe[column][i] = item

In [8]:
def remove_comma(dataframe, column_list):
    """
    This function removes the comma from a value in a column.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns less any commas that might appear.    
    """  
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item.split(',')) > 1:
                one, two = item.split(',')
                dataframe[column][i] = one + two

In [9]:
def adjust_float(dataframe, column_list):
    """
    This function adjusts values in a column to be workable with the 'update_float_dtype' function.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns with objects with a decimal point value behind it.    
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item) < 3:
                dataframe[column][i] = item + '.0'
            else:
                dataframe[column][i] = item

In [10]:
def update_percent(dataframe, column):
    """
    This function updates a column with a percent sign so it can be manipulated into a float with the 
    'update_float_dtype' function.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed.     
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip('%'))
    for i in range(len(dataframe[column])):
        item = dataframe[column][i]
        if len(item) < 3:
            dataframe[column][i] = item + '.0'
        else:
            dataframe[column][i] = item

In [11]:
def replace_dash(dataframe, column_list, dash):
    """
    This function replaces a - in a dataframe so datatypes can then be updated.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        dash: set to a value in a specific column in the dataframe.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed. 
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == dash:
                dataframe[column][i] = '0'
            else:
                dataframe[column][i] = item

## Read in Collected Data

In [193]:
df = pd.read_pickle('player_stats')
fantasy_pts = pd.read_pickle('fantasy_weeks')
defense = pd.read_pickle('defense_data')
kicking = pd.read_pickle('kicking_data')

## Merge Last Week Fantasy Pts with this Week

In [194]:
no_games = 9

In [195]:
lastwk = pd.read_pickle('fantweeks_1_' + str(no_games - 1))
lastwk.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8
0,Russell Wilson,SEA,QB,212.0,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7
1,Patrick Mahomes II,KC,QB,208.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6
2,Kyler Murray,ARI,QB,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN
3,Josh Allen,BUF,QB,184.6,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5
4,Tom Brady,TB,QB,174.3,22.5,9.7,23.9,33.5,14.1,14.6,36.9,19.1


In [196]:
fantasy_pts = pd.merge(fantasy_pts, lastwk, 'left', on='Player')
fantasy_pts.head()

,Player,Team_x,Position_x,TTL_x,Week_9,Team_y,Position_y,TTL_y,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8
0,Kyler Murray,ARI,QB,240.4,37.9,ARI,QB,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN
1,Patrick Mahomes II,KC,QB,238.9,30.9,KC,QB,208.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6
2,Russell Wilson,SEA,QB,236.1,24.1,SEA,QB,212.0,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7
3,Josh Allen,BUF,QB,220.6,36.0,BUF,QB,184.6,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5
4,Aaron Rodgers,GB,QB,187.6,28.9,GB,QB,158.7,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5


In [197]:
fantasy_pts.drop(columns=['Team_y', 'Position_y', 'TTL_y'], inplace=True)
fantasy_pts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position', 'TTL_x': 'TTL'}, inplace = True)
fantasy_pts.columns

Index(['Player', 'Team', 'Position', 'TTL', 'Week_9', 'Week_1', 'Week_2',
       'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8'],
      dtype='object')

In [198]:
fantasy_pts = fantasy_pts[['Player', 'Team', 'Position', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 
                           'Week_6', 'Week_7', 'Week_8', 'Week_9']]
fantasy_pts.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Kyler Murray,ARI,QB,240.4,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9
1,Patrick Mahomes II,KC,QB,238.9,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9
2,Russell Wilson,SEA,QB,236.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1
3,Josh Allen,BUF,QB,220.6,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0
4,Aaron Rodgers,GB,QB,187.6,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9


In [199]:
print(fantasy_pts.shape)

(651, 13)


### Pickle Combined DataFrame for Next Week

In [200]:
fantasy_pts.to_pickle('fantweeks_1_' + str(no_games))

## Merge Offensive Players & Weekly Fantasy Pts

In [201]:
print(df.shape)
df.head()

(556, 28)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Dalvin Cook,MIN,RB,25,7,7,0,0,0,0,0,144,858,5.96,12,21,16,173,10.81,1,1,1,13,3,,193.1,1,1
1,Patrick Mahomes,KAN,QB,25,9,9,220,329,2687,25,1,36,165,4.58,2,0,0,0,,0,1,0,2,,2,239.0,1,2
2,Alvin Kamara,NOR,RB,25,8,6,0,0,0,0,0,96,471,4.91,5,72,60,565,9.42,3,1,0,8,,,181.6,2,3
3,Kyler Murray,ARI,QB,23,8,8,190,279,2130,16,7,76,543,7.14,8,0,0,0,,0,4,2,8,,,240.5,2,4
4,Russell Wilson,SEA,QB,32,8,8,211,297,2541,28,8,37,265,7.16,1,0,0,0,,0,3,2,1,,1,236.1,3,5


In [202]:
print(fantasy_pts.shape)
fantasy_pts.head()

(651, 13)


,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Kyler Murray,ARI,QB,240.4,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9
1,Patrick Mahomes II,KC,QB,238.9,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9
2,Russell Wilson,SEA,QB,236.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1
3,Josh Allen,BUF,QB,220.6,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0
4,Aaron Rodgers,GB,QB,187.6,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9


In [203]:
check_names(df, fantasy_pts)

In [204]:
name_update(df, 'Player')
name_update(fantasy_pts, 'Player')

In [205]:
offense = pd.merge(df, fantasy_pts, 'left', on='Player')

In [206]:
offense.drop(columns = ['Team_y', 'Position_y'], inplace = True)
offense.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)

In [207]:
print(offense.shape)
offense.head(10)

(556, 38)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Dalvin Cook,MIN,RB,25,7,7,0,0,0,0,0,144,858,5.96,12,21,16,173,10.81,1,1,1,13,3,,193.1,1,1,185.1,20.8,15.1,23.9,26.6,14.9,NaN,NaN,46.6,37.2
1,Patrick Mahomes,KAN,QB,25,9,9,220,329,2687,25,1,36,165,4.58,2,0,0,0,,0,1,0,2,,2,239.0,1,2,238.9,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9
2,Alvin Kamara,NOR,RB,25,8,6,0,0,0,0,0,96,471,4.91,5,72,60,565,9.42,3,1,0,8,,,181.6,2,3,151.6,18.7,29.4,31.7,17.9,11.9,NaN,14.8,16.3,10.9
3,Kyler Murray,ARI,QB,23,8,8,190,279,2130,16,7,76,543,7.14,8,0,0,0,,0,4,2,8,,,240.5,2,4,240.4,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9
4,Russell Wilson,SEA,QB,32,8,8,211,297,2541,28,8,37,265,7.16,1,0,0,0,,0,3,2,1,,1,236.1,3,5,236.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1
5,Derrick Henry,TEN,RB,26,8,8,0,0,0,0,0,182,843,4.63,8,18,10,81,8.10,0,0,0,8,,,145.4,3,6,140.4,13.1,8.4,25.0,NaN,18.3,38.4,13.2,17.2,6.8
6,Tyreek Hill,KAN,WR,26,9,9,0,0,0,0,0,8,62,7.75,1,72,44,650,14.77,9,1,0,10,,,153.2,1,7,131.2,10.6,16.8,16.2,12.4,15.3,2.5,11.5,21.8,24.1
7,Travis Kelce,KAN,TE,31,9,9,0,0,0,0,0,0,0,,0,80,58,769,13.26,6,1,1,6,,,139.9,1,8,110.9,11.0,15.0,8.7,7.0,16.8,16.5,3.1,16.9,15.9
8,D.K. Metcalf,SEA,WR,23,8,8,0,0,0,0,0,0,0,,0,68,43,788,18.33,8,1,1,8,,,146.3,2,9,124.8,15.5,15.2,15.0,10.6,21.3,NaN,2.3,28.1,16.8
9,Josh Allen,BUF,QB,24,9,9,217,315,2587,19,5,65,241,3.71,5,0,0,0,,0,4,4,5,,,220.6,4,10,220.6,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0


In [208]:
offense.isnull().sum()

Player                     0
Team                       0
Position                   0
Age                        0
Games                      0
GamesStarted               0
CompletedPasses            0
PassesAttempted            0
PassingYds                 0
PassingTDs                 0
Interceptions              0
RushingAttempts            0
RushingYds                 0
RushingYdspAtt             0
RushingTDs                 0
Targeted                   0
Receptions                 0
ReceivingYds               0
YdspReception              0
ReceivingTDs               0
Fumbles                    0
LostFumbles                0
TtlTDs                     0
TwoPTConversions           0
TwoPTConversionPasses      0
FDFantasyPts               0
PositionRank               0
OverallRank                0
TTL                       17
Week_1                   149
Week_2                   151
Week_3                   152
Week_4                   178
Week_5                   208
Week_6        

In [209]:
offense.fillna('0.0', inplace = True)

In [210]:
off_integers = ['Age', 'Games', 'GamesStarted', 'CompletedPasses', 'PassesAttempted', 'PassingYds', 'PassingTDs', 
            'Interceptions', 'RushingAttempts', 'RushingYds', 'RushingTDs', 'Targeted', 'Receptions', 
            'ReceivingYds', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs']

off_floats = ['RushingYdspAtt', 'YdspReception', 'FDFantasyPts', 'TTL','Week_1', 'Week_2', 'Week_3', 'Week_4', 
              'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9']

In [211]:
update_int_dtype(offense, off_integers)

In [212]:
two_pts = ['TwoPTConversions', 'TwoPTConversionPasses']
fill_blanks(offense, two_pts)
update_int_dtype(offense, two_pts)

In [213]:
update_float_dtype(offense, off_floats)

In [214]:
offense.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Dalvin Cook,MIN,RB,25,7,7,0,0,0,0,0,144,858,14.6,12,21,16,173,18.1,1,1,1,13,3,0,193.1,1,1,185.1,20.8,15.1,23.9,26.6,14.9,0.0,0.0,46.6,37.2
1,Patrick Mahomes,KAN,QB,25,9,9,220,329,2687,25,1,36,165,9.8,2,0,0,0,0.0,0,1,0,2,0,2,239.0,1,2,238.9,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9
2,Alvin Kamara,NOR,RB,25,8,6,0,0,0,0,0,96,471,13.1,5,72,60,565,13.2,3,1,0,8,0,0,181.6,2,3,151.6,18.7,29.4,31.7,17.9,11.9,0.0,14.8,16.3,10.9
3,Kyler Murray,ARI,QB,23,8,8,190,279,2130,16,7,76,543,8.4,8,0,0,0,0.0,0,4,2,8,0,0,240.5,2,4,240.4,27.3,33.1,24.7,23.1,27.3,28.9,38.1,0.0,37.9
4,Russell Wilson,SEA,QB,32,8,8,211,297,2541,28,8,37,265,8.6,1,0,0,0,0.0,0,3,2,1,0,1,236.1,3,5,236.1,31.8,34.4,36.8,21.9,25.5,0.0,32.9,28.7,24.1


In [215]:
team_dict = {'LAR': 'Los Angeles Rams', 
             'SEA': 'Seattle Seahawks', 
             'BAL': 'Baltimore Ravens', 
             'KAN': 'Kansas City Chiefs', 
             'JAX': 'Jacksonville Jaguars', 
             'DET': 'Detroit Lions', 
             'PIT': 'Pittsburgh Steelers', 
             'MIN': 'Minnesota Vikings', 
             'CHI': 'Chicago Bears', 
             'ATL': 'Atlanta Falcons', 
             'BUF': 'Buffalo Bills', 
             'GNB': 'Green Bay Packers', 
             'MIA': 'Miami Dolphins', 
             'CLE': 'Cleveland Browns', 
             'CAR': 'Carolina Panthers', 
             'DAL': 'Dallas Cowboys', 
             'ARI': 'Arizona Cardinals', 
             'HOU': 'Houston Texans', 
             'NYG': 'New York Giants', 
             'WAS': 'Washington Football Team', 
             'IND': 'Indianapolis Colts', 
             'LVR': 'Las Vegas Raiders', 
             'DEN': 'Denver Broncos', 
             'TEN': 'Tennessee Titans', 
             'NWE': 'New England Patriots', 
             'CIN': 'Cincinnati Bengals', 
             'NYJ': 'New York Jets', 
             'LAC': 'Los Angeles Chargers', 
             'PHI': 'Philadelphia Eagles', 
             'TAM': 'Tampa Bay Buccaneers', 
             'NOR': 'New Orleans Saints', 
             'SFO': 'San Francisco 49ers'}

team_names = pd.DataFrame()
team_names['Team'] = list(team_dict.keys())
team_names['Long_Name'] = list(team_dict.values())
team_names.head()

,Team,Long_Name
0,LAR,Los Angeles Rams
1,SEA,Seattle Seahawks
2,BAL,Baltimore Ravens
3,KAN,Kansas City Chiefs
4,JAX,Jacksonville Jaguars


In [216]:
offense.dtypes

Player                    object
Team                      object
Position                  object
Age                        int64
Games                      int64
GamesStarted               int64
CompletedPasses            int64
PassesAttempted            int64
PassingYds                 int64
PassingTDs                 int64
Interceptions              int64
RushingAttempts            int64
RushingYds                 int64
RushingYdspAtt           float64
RushingTDs                 int64
Targeted                   int64
Receptions                 int64
ReceivingYds               int64
YdspReception            float64
ReceivingTDs               int64
Fumbles                    int64
LostFumbles                int64
TtlTDs                     int64
TwoPTConversions           int64
TwoPTConversionPasses      int64
FDFantasyPts             float64
PositionRank              object
OverallRank               object
TTL                      float64
Week_1                   float64
Week_2    

## Defense Stats

In [217]:
defense.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Baltimore Ravens,8,142,512,5.1,"2,599",180,814,180,6,308,45.9,154,71.6,"1,785",10,18,14,70.9%,50,4,10,24
1,Los Angeles Rams,8,152,493,4.7,"2,335",192,758,192,6,276,44.9,169,68.7,"1,577",9,23,14,64.6%,64,6,4,25
2,Indianapolis Colts,8,160,481,4.8,"2,320",202,669,202,6,259,50.6,125,79.2,"1,651",10,21,14,66.8%,112,11,2,20
3,Miami Dolphins,8,161,531,5.8,"3,074",215,"1,054",215,10,296,58,146,86.2,"2,020",11,25,17,65.8%,77,7,7,20
4,Pittsburgh Steelers,8,161,500,5.1,"2,539",200,822,200,5,268,51.9,122,91.8,"1,717",14,22,13,64.8%,114,11,4,32


In [218]:
def_fantasy = fantasy_pts[fantasy_pts['Position'] == 'DST']

In [219]:
defense_df = pd.merge(defense, def_fantasy, 'left', left_on = 'Team', right_on = 'Player')

In [220]:
defense_df.head()

,Team_x,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,Player,Team_y,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Baltimore Ravens,8,142,512,5.1,"2,599",180,814,180,6,308,45.9,154,71.6,"1,785",10,18,14,70.9%,50,4,10,24,Baltimore Ravens,BAL,DST,94.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN,4.0,14.0
1,Los Angeles Rams,8,152,493,4.7,"2,335",192,758,192,6,276,44.9,169,68.7,"1,577",9,23,14,64.6%,64,6,4,25,Los Angeles Rams,LAR,DST,58.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,NaN
2,Indianapolis Colts,8,160,481,4.8,"2,320",202,669,202,6,259,50.6,125,79.2,"1,651",10,21,14,66.8%,112,11,2,20,Indianapolis Colts,IND,DST,88.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN,15.0,5.0
3,Miami Dolphins,8,161,531,5.8,"3,074",215,"1,054",215,10,296,58,146,86.2,"2,020",11,25,17,65.8%,77,7,7,20,Miami Dolphins,MIA,DST,77.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN,17.0,8.0
4,Pittsburgh Steelers,8,161,500,5.1,"2,539",200,822,200,5,268,51.9,122,91.8,"1,717",14,22,13,64.8%,114,11,4,32,Pittsburgh Steelers,PIT,DST,81.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0


In [221]:
defense_df.drop(columns=['Player', 'Team_y', 'Position'], inplace=True)
defense_df.rename(columns={'Team_x': 'Team'}, inplace=True)

In [222]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Baltimore Ravens,8,142,512,5.1,"2,599",180,814,180,6,308,45.9,154,71.6,"1,785",10,18,14,70.9%,50,4,10,24,94.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN,4.0,14.0
1,Los Angeles Rams,8,152,493,4.7,"2,335",192,758,192,6,276,44.9,169,68.7,"1,577",9,23,14,64.6%,64,6,4,25,58.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,NaN
2,Indianapolis Colts,8,160,481,4.8,"2,320",202,669,202,6,259,50.6,125,79.2,"1,651",10,21,14,66.8%,112,11,2,20,88.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN,15.0,5.0
3,Miami Dolphins,8,161,531,5.8,"3,074",215,"1,054",215,10,296,58,146,86.2,"2,020",11,25,17,65.8%,77,7,7,20,77.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN,17.0,8.0
4,Pittsburgh Steelers,8,161,500,5.1,"2,539",200,822,200,5,268,51.9,122,91.8,"1,717",14,22,13,64.8%,114,11,4,32,81.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0


In [223]:
defense_df.isnull().sum()

Team                         0
GP                           0
Ttl_Pts_Allowed              0
Ttl_Offense_Plays_Allowed    0
Yds_p_Play                   0
Ttl_Yds                      0
Rushing_Att                  0
Rushing_Yds                  0
Rushing_Yds_p_Att            0
Rushing_TDs                  0
Passing_Att                  0
Passing_Yds_p_Att            0
Completions                  0
Yds_p_Completion             0
Passing_Yds                  0
Passing_TDs                  0
RZ_Att                       0
RZ_TD                        0
RZ_Percent                   0
Ttl_Turnovers                0
Interceptions                0
Fumbles                      0
Sacks                        0
TTL                          0
Week_1                       0
Week_2                       0
Week_3                       0
Week_4                       0
Week_5                       4
Week_6                       4
Week_7                       4
Week_8                       4
Week_9  

In [224]:
defense_df

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Baltimore Ravens,8,142,512,5.1,"2,599",180,814,180,6,308,45.9,154,71.6,"1,785",10,18,14,70.9%,50,4,10,24,94.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN,4.0,14.0
1,Los Angeles Rams,8,152,493,4.7,"2,335",192,758,192,6,276,44.9,169,68.7,"1,577",9,23,14,64.6%,64,6,4,25,58.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,NaN
2,Indianapolis Colts,8,160,481,4.8,"2,320",202,669,202,6,259,50.6,125,79.2,"1,651",10,21,14,66.8%,112,11,2,20,88.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN,15.0,5.0
3,Miami Dolphins,8,161,531,5.8,"3,074",215,"1,054",215,10,296,58,146,86.2,"2,020",11,25,17,65.8%,77,7,7,20,77.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN,17.0,8.0
4,Pittsburgh Steelers,8,161,500,5.1,"2,539",200,822,200,5,268,51.9,122,91.8,"1,717",14,22,13,64.8%,114,11,4,32,81.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0
5,Arizona Cardinals,8,180,541,5.5,"2,961",220,"1,009",220,5,299,53.2,173,83.1,"1,952",14,30,15,50.5%,64,6,4,22,49.0,4.0,9.0,4.0,1.0,7.0,15.0,7.0,NaN,2.0
6,Kansas City Chiefs,9,183,589,5.4,"3,187",262,"1,246",262,9,308,56.1,150,89.1,"1,941",11,24,18,69.4%,95,9,5,19,73.0,7.0,5.0,7.0,20.0,-1.0,3.0,18.0,7.0,1.0
7,Washington Football Team,8,188,498,5,"2,514",240,"1,029",240,9,231,51.3,127,78.7,"1,485",11,23,13,56.2%,81,8,1,27,52.0,15.0,4.0,1.0,4.0,2.0,4.0,17.0,NaN,5.0
8,Chicago Bears,9,190,588,5.1,"3,016",254,"1,053",254,6,314,56.6,161,97,"1,963",10,31,13,46%,54,5,4,20,52.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0,1.0,4.0
9,New England Patriots,8,194,456,6.1,"2,823",234,"1,048",234,8,211,67.8,117,102.4,"1,775",12,25,15,57.8%,104,10,4,11,53.0,11.0,6.0,15.0,4.0,NaN,7.0,4.0,3.0,3.0


In [225]:
defense_df.fillna('0.0', inplace=True)

In [226]:
def_int = ['GP', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 
           'Rushing_TDs', 'Passing_Att', 'Completions', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD', 
           'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks']

def_floats = ['Yds_p_Play', 'TTL', 'Week_1', 
              'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9']

# 'Rushing_Yds_p_Att', 'Passing_Yds_p_Att', 'Yds_p_Completion',

In [227]:
remove_comma(defense_df, def_int)

In [228]:
update_int_dtype(defense_df, def_int)

In [229]:
adjust_float(defense_df, def_floats)

In [230]:
defense_df['Rushing_Yds_p_Att'] = round(defense_df['Rushing_Yds']/defense_df['Rushing_Att'], 2)
defense_df['Passing_Yds_p_Att'] = round(defense_df['Passing_Yds']/defense_df['Passing_Att'], 2)
defense_df['Yds_p_Completion'] = round(defense_df['Passing_Yds']/defense_df['Completions'], 2)

In [231]:
update_float_dtype(defense_df, def_floats)

In [232]:
update_percent(defense_df, 'RZ_Percent')

In [233]:
rz = ['RZ_Percent']
update_float_dtype(defense_df, rz)

In [234]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Baltimore Ravens,8,142,512,5.1,2599,180,814,4.5,6,308,5.8,154,11.6,1785,10,18,14,70.9,50,4,10,24,94.0,15.0,15.0,1.0,6.0,26.0,7.0,0.0,4.0,14.0
1,Los Angeles Rams,8,152,493,4.7,2335,192,758,4.0,6,276,5.7,169,9.3,1577,9,23,14,64.6,64,6,4,25,58.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,0.0
2,Indianapolis Colts,8,160,481,4.8,2320,202,669,3.3,6,259,6.4,125,13.2,1651,10,21,14,66.8,112,11,2,20,88.0,4.0,15.0,26.0,7.0,5.0,4.0,0.0,15.0,5.0
3,Miami Dolphins,8,161,531,5.8,3074,215,1054,4.9,10,296,6.8,146,13.8,2020,11,25,17,65.8,77,7,7,20,77.0,2.0,2.0,12.0,3.0,12.0,15.0,0.0,17.0,8.0
4,Pittsburgh Steelers,8,161,500,5.1,2539,200,822,4.1,5,268,6.4,122,14.1,1717,14,22,13,64.8,114,11,4,32,81.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0


In [235]:
defense_df.dtypes

Team                          object
GP                             int64
Ttl_Pts_Allowed                int64
Ttl_Offense_Plays_Allowed      int64
Yds_p_Play                   float64
Ttl_Yds                        int64
Rushing_Att                    int64
Rushing_Yds                    int64
Rushing_Yds_p_Att            float64
Rushing_TDs                    int64
Passing_Att                    int64
Passing_Yds_p_Att            float64
Completions                    int64
Yds_p_Completion             float64
Passing_Yds                    int64
Passing_TDs                    int64
RZ_Att                         int64
RZ_TD                          int64
RZ_Percent                   float64
Ttl_Turnovers                  int64
Interceptions                  int64
Fumbles                        int64
Sacks                          int64
TTL                          float64
Week_1                       float64
Week_2                       float64
Week_3                       float64
W

## Kicking Stats

In [236]:
kicking.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS
0,Younghoe Koo,K,ATL,8,22,95.5,54,0-0,5-5,6-6,7-8,3-3,21,85.7,81
1,Wil Lutz,K,NO,8,17,94.1,53,0-0,3-4,8-8,4-4,1-1,28,100.0,76
2,Joey Slye,K,CAR,9,24,79.2,48,0-0,8-8,6-6,5-6,0-4,21,90.5,76
3,Tyler Bass,K,BUF,9,22,72.7,53,1-1,4-4,4-7,6-7,1-3,27,96.3,74
4,Randy Bullock,K,CIN,8,20,90.0,55,0-0,3-3,7-8,6-7,2-2,20,100.0,74


In [237]:
kicking_df = pd.merge(kicking, fantasy_pts, left_on = 'PLAYER', right_on = 'Player')
kicking_df.drop(columns = ['Player', 'Team', 'Position'], inplace = True)
kicking_df

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Younghoe Koo,K,ATL,8,22,95.5,54,0-0,5-5,6-6,7-8,3-3,21,85.7,81,94.0,9.0,17.0,8.0,NaN,13.0,20.0,2.0,13.0,12.0
1,Wil Lutz,K,NO,8,17,94.1,53,0-0,3-4,8-8,4-4,1-1,28,100.0,76,82.0,10.0,6.0,13.0,5.0,15.0,NaN,11.0,14.0,8.0
2,Joey Slye,K,CAR,9,24,79.2,48,0-0,8-8,6-6,5-6,0-4,21,90.5,76,81.0,12.0,5.0,15.0,7.0,11.0,11.0,7.0,5.0,8.0
3,Tyler Bass,K,BUF,9,22,72.7,53,1-1,4-4,4-7,6-7,1-3,27,96.3,74,82.0,9.0,7.0,5.0,6.0,5.0,6.0,23.0,6.0,15.0
4,Randy Bullock,K,CIN,8,20,90.0,55,0-0,3-3,7-8,6-7,2-2,20,100.0,74,84.0,10.0,13.0,12.0,17.0,3.0,12.0,10.0,7.0,NaN
5,Daniel Carlson,K,LV,8,19,89.5,54,0-0,7-7,5-5,2-4,3-3,24,95.8,74,82.0,12.0,12.0,8.0,13.0,11.0,NaN,9.0,10.0,7.0
6,Ryan Succop,K,TB,9,18,88.9,50,0-0,3-3,7-7,5-6,1-2,27,96.3,74,81.0,5.0,7.0,9.0,8.0,14.0,10.0,9.0,15.0,4.0
7,Harrison Butker,K,KC,9,17,88.2,58,1-1,3-3,7-7,1-3,3-3,33,81.8,72,79.0,10.0,13.0,4.0,8.0,6.0,8.0,14.0,5.0,11.0
8,Jason Sanders,K,MIA,8,17,100.0,56,0-0,5-5,2-2,6-6,4-4,21,100.0,72,86.0,4.0,11.0,7.0,18.0,22.0,6.0,NaN,4.0,14.0
9,Justin Tucker,K,BAL,8,16,93.8,55,0-0,2-2,3-3,8-8,2-3,26,100.0,71,83.0,9.0,16.0,9.0,8.0,10.0,16.0,NaN,8.0,7.0


In [238]:
dash = kicking_df['LNG'][38]
dash

'—'

In [239]:
kicking_df.fillna('0.0', inplace = True)

In [240]:
kick_int = ['GP', 'FGM_A', 'LNG', 'XPM_A', 'PTS']

kick_float = ['FG%', 'XP%', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 
              'Week_9']


In [241]:
replace_dash(kicking_df, kick_int, dash)

In [242]:
replace_dash(kicking_df, kick_float, dash)

In [243]:
update_int_dtype(kicking_df, kick_int)

In [244]:
adjust_float(kicking_df, kick_float)

In [245]:
update_float_dtype(kicking_df, kick_float)

In [246]:
kicking_df.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9
0,Younghoe Koo,K,ATL,8,22,95.5,54,0-0,5-5,6-6,7-8,3-3,21,85.7,81,94.0,9.0,17.0,8.0,0.0,13.0,20.0,2.0,13.0,12.0
1,Wil Lutz,K,NO,8,17,94.1,53,0-0,3-4,8-8,4-4,1-1,28,100.0,76,82.0,10.0,6.0,13.0,5.0,15.0,0.0,11.0,14.0,8.0
2,Joey Slye,K,CAR,9,24,79.2,48,0-0,8-8,6-6,5-6,0-4,21,90.5,76,81.0,12.0,5.0,15.0,7.0,11.0,11.0,7.0,5.0,8.0
3,Tyler Bass,K,BUF,9,22,72.7,53,1-1,4-4,4-7,6-7,1-3,27,96.3,74,82.0,9.0,7.0,5.0,6.0,5.0,6.0,23.0,6.0,15.0
4,Randy Bullock,K,CIN,8,20,90.0,55,0-0,3-3,7-8,6-7,2-2,20,100.0,74,84.0,10.0,13.0,12.0,17.0,3.0,12.0,10.0,7.0,0.0


In [247]:
kicking_df.dtypes

PLAYER       object
POSITION     object
TEAM         object
GP            int64
FGM_A         int64
FG%         float64
LNG           int64
_1_19        object
_20_29       object
_30_39       object
_40_49       object
OVER50       object
XPM_A         int64
XP%         float64
PTS           int64
TTL         float64
Week_1      float64
Week_2      float64
Week_3      float64
Week_4      float64
Week_5      float64
Week_6      float64
Week_7      float64
Week_8      float64
Week_9      float64
dtype: object

## Pickle Cleaned DataFrames

In [248]:
offense.to_pickle('players')
team_names.to_pickle('long_names')
defense_df.to_pickle('defense')
kicking_df.to_pickle('kicking')